In [1]:
!pip install openai
!pip install pillow



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import openai
import json
from datetime import datetime, timedelta
import os
from PIL import Image, ImageDraw, ImageFont
import textwrap

In [3]:
topic = "weekend plans"

#Prompts
system_prompt = """
write the text at an 3rd grade reading and writing level in simple language. The text you will rewrite will follow the colon (:) at the end. If there is no colon then rewrite your previous output in the conversation before this prompt, but only if there is no text after the colon.  You MUST keep the formatting and header formatting of the original.

You must avoid technical jargon and business jargon so write in a casual and direct way without losing any of the key concepts in the text you are rewriting. Key concepts are defined as powerful statements, emotional sentences, or sentences containing industry terms or proper nouns.

Constraints:
* Remove emojis.
* Never start with a question. Instead use an interest piquing personal statement.
* Make sure there are smooth transitions between sentences.
* Remove all metaphors and analogies.
* Keep the same identical formatting of the existing text.
 * Remove any formatting like H1 (#), H2 (##), bold (**), etc or any other markdown formatting. Just use newlines and line breaks.

When writing, please do not use the following words or phrases or any words similar to the following in any of the content:
realm, landscape, game-changing, in conclusion, firstly, secondly, lastly, delve, in light of, not to mention, to say nothing of, by the same token, moreover, as well as, furthermore, therefore, top-notch, get ready, buckle up, switching gears, dive in, now let’s move on, in conclusion, demystifying, delve, ever-evolving, innovative solution, let’s dive in, let’s delve, folks, picturesque, unleash, dive in, voyage, picture this, say goodbye to, according to my database, treasure trove, let’s begin this journey, let’s delve, go deeper, explore now, navigating, delve into, shed light, gone are the days.

When writing, you may use these words as needed or any words similar to the following, but never in the first line of any paragraph: first, second, third, important, equally, identically, uniquely, together with, likewise, comparatively, correspondingly, similarly, additionally, explore, crucial, whimsical, embrace, freedom, essential, imperative, important, whilst, explore, discover, elevate, solace.
"""  

# Define an initial prompt
prompt = f"""You are generating a friendly conversation between two people, Person 1 and Person 2, about {topic}. Each person should respond in 1 short sentence. Format each message in JSON format as follows: {{\"speaker\": \"Person X\", \"message\": \"Text\", \"timestamp\": \"Time\"}}. Each response should be a single JSON object.

Example:
[
    {{   
        \"speaker\": \"Person 2\",
        \"message\": \"Sounds good. Looking forward to it!\",
        \"timestamp\": \"01:44 PM\"
    }},
    {{
        \"speaker\": \"Person 1\",
        \"message\": \"Great, me too. Talk to you soon!\",
        \"timestamp\": \"01:46 PM\"
    }},
    {{
        \"speaker\": \"Person 2\",
        \"message\": \"Sure, see you soon. Have a nice day!\",
        \"timestamp\": \"01:48 PM\"
    }},
    {{
        \"speaker\": \"Person 1\",
        \"message\": \"You too, have a great day!\",
        \"timestamp\": \"01:50 PM\"
    }}
]
"""

In [4]:
# Set up your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Function to simulate a conversation between two personas
def generate_conversation(topic, num_turns=5):
    conversation = []
    current_time = datetime.now()

    for turn in range(num_turns * 2):  # Multiply by 2 for alternating turns
        # Determine which person is speaking
        speaker = "Person 1" if turn % 2 == 0 else "Person 2"

        # Create a conversation prompt by including the dialogue so far
        conversation_prompt = f"{prompt}\n\nCurrent conversation:\n" + "\n".join([json.dumps(line) for line in conversation])

        # Generate the response using the updated API
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": conversation_prompt}
            ]
        )

        try:
            # Parse the JSON response - it will be a list of messages
            messages = json.loads(response.choices[0].message.content)
            
            # Take only the first num_turns * 2 messages (or all if fewer)
            conversation = messages[:num_turns * 2]
            
            # Update timestamps to be sequential from current time
            for i, msg in enumerate(conversation):
                msg["timestamp"] = (current_time + timedelta(minutes=i * 2)).strftime("%I:%M %p")
                
            return conversation
            
        except json.JSONDecodeError:
            print("Error: Could not parse the response as JSON")
            return []

# Generate a conversation on a given topic
conversation_data = generate_conversation(topic, num_turns=5)

# Print the structured JSON conversation
print(json.dumps(conversation_data, indent=4))


[
    {
        "speaker": "Person 1",
        "message": "What's your plan for the weekend?",
        "timestamp": "03:30 PM"
    },
    {
        "speaker": "Person 2",
        "message": "I'm thinking of going to a movie.",
        "timestamp": "03:32 PM"
    },
    {
        "speaker": "Person 1",
        "message": "That sounds like fun!",
        "timestamp": "03:34 PM"
    },
    {
        "speaker": "Person 2",
        "message": "Yes, it should be. What about you?",
        "timestamp": "03:36 PM"
    },
    {
        "speaker": "Person 1",
        "message": "I'll probably just relax at home.",
        "timestamp": "03:38 PM"
    },
    {
        "speaker": "Person 2",
        "message": "That sounds nice too. Maybe we can hang out next time.",
        "timestamp": "03:40 PM"
    },
    {
        "speaker": "Person 1",
        "message": "Yes, let's plan that. Enjoy your movie!",
        "timestamp": "03:42 PM"
    },
    {
        "speaker": "Person 2",
        "message": "T

### Image Generation with PIL

In [7]:
# Sample conversation data structure
# conversation_data = [
#     {"sender": "Shaima", "message": "Hello! 👋\nHow it's going?", "timestamp": "20:21 PM", "status": "sent"},
#     {"sender": "You", "message": "Hey!", "timestamp": "20:21 PM", "status": "read"},
#     {"sender": "Shaima", "message": "Hello! 👋\nHow it's going?", "timestamp": "20:21 PM", "status": "sent"},
#     {"sender": "You", "message": "Hey there!\nHow it's going?", "timestamp": "20:21 PM", "status": "read"},
#     {"date_divider": "Mon, 34 Aug"},
#     {"sender": "You", "message": "Hey there!\nHow’s everything with you?", "timestamp": "20:21 PM", "status": "read"}
# ]

# Parameters for the chat image
width, height = 500, 1000
background_color = "#F0F0F0"  # Lightest grey in Pillow's web-safe colors
sender_color = "lightgrey"
receiver_color = "white"
text_color = "black"
timestamp_color = "grey"
date_divider_color = "orange"

# Font settings
# font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"  # Replace with your font path
# font_size = 20
# timestamp_font_size = 14

# Load fonts
# font = ImageFont.truetype(font_path, font_size)
# timestamp_font = ImageFont.truetype(font_path, timestamp_font_size)

# Initialize image
img = Image.new("RGB", (width, height), color=background_color)
draw = ImageDraw.Draw(img)

# Function to draw a chat bubble
def draw_bubble(draw, text, sender=True, timestamp="", status=None, y_position=0):
    # Determine bubble size
    max_text_width = 280
    wrapped_text = textwrap.fill(text, width=30)
    # Deprecated textsize replace with textbbox
    bbox = draw.textbbox((0, 0), wrapped_text)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    bubble_width = min(max_text_width, text_width + 20)
    bubble_height = text_height + 40
    

    # Set position and color
    x_position = 20 if sender else width - bubble_width - 20
    bubble_color = sender_color if sender else receiver_color

    # Draw bubble
    draw.rounded_rectangle((x_position, y_position, x_position + bubble_width, y_position + bubble_height), radius=15, fill=bubble_color)

    # Draw text
    draw.multiline_text((x_position + 10, y_position + 10), wrapped_text, fill=text_color)

    # Draw timestamp
    timestamp_position = (x_position + bubble_width - 50, y_position + bubble_height - 20)
    draw.text(timestamp_position, timestamp, fill=timestamp_color)

    # Draw status icon (e.g., check mark for read status)
    if status == "read":
        status_position = (x_position + bubble_width - 30, y_position + bubble_height - 20)
        draw.text(status_position, "✓", fill=timestamp_color)

    # Return the new y_position for the next message
    return y_position + bubble_height + 20

# Function to draw date divider
def draw_date_divider(draw, text, y_position):
    # Replace textsize with textbbox
    bbox = draw.textbbox((0, 0), text)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    divider_x_position = (width - text_width) // 2
    draw.text((divider_x_position, y_position), text, fill=date_divider_color)
    return y_position + text_height + 20

# Draw the chat conversation
current_y = 50  # Starting y position

for i in range(len(conversation_data)):
    # Initialize new image for each progressive conversation
    img = Image.new("RGB", (width, height), color=background_color)
    draw = ImageDraw.Draw(img)
    
    current_y = 50  # Reset starting y position
    
    # Draw conversation up to current message
    for item in conversation_data[:i+1]:
        sender = item["speaker"] == "Person 2"
        current_y = draw_bubble(
            draw, 
            item["message"], 
            sender=sender, 
            timestamp=item["timestamp"], 
            y_position=current_y
        )
    
    # Generate a unique timestamp for the folder
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"conversation_images_{timestamp}"
    
    # Create output directory if it doesn't exist
    os.makedirs(folder_name, exist_ok=True)
    
    # Save the progressive conversation in the uniquely named folder
    img.save(f"{folder_name}/conversation_{i+1}.png")


### Movie Maker

In [8]:
!pip install moviepy
!pip install --upgrade decorator==4.4.2
!pip install ffmpeg --upgrade


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os
import moviepy.editor as mp
from moviepy.editor import ImageSequenceClip


In [11]:
# Directory where images are stored
output_video = f"{folder_name}/chat_conversation.mp4"

# Get list of image files in order
image_files = sorted([os.path.join(folder_name, img) for img in os.listdir(folder_name) if img.endswith(".png")])

# Define video parameters
fps = 0.5  # Frames per second; adjust based on how fast you want the conversation to flow
duration = len(image_files) / fps

# Create the video clip
clip = ImageSequenceClip(image_files, fps=fps)
clip.set_duration(duration)

# Save the video
clip.write_videofile(output_video, codec="libx264")


Moviepy - Building video conversation_images_20241110_153142/chat_conversation.mp4.
Moviepy - Writing video conversation_images_20241110_153142/chat_conversation.mp4



Moviepy - Done !
Moviepy - video ready conversation_images_20241110_153142/chat_conversation.mp4
